In [43]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics

In [44]:
using Pkg
Pkg.add("PyCall")

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [45]:
# Read the data
stations_info = CSV.read("../data/stations/station_information.csv", Tables.matrix, header=1);
#stations_info

449×15 Matrix{Any}:
   3  "classic"  …  "B32006"  42.3401   10
   4  "classic"     "C32000"  42.3454   10
   5  "classic"     "B32012"  42.3418   10
   6  "classic"     "D32000"  42.3613   10
   7  "classic"     "A32000"  42.3534   10
   8  "classic"  …  "A32001"  42.3533   10
   9  "classic"     "A32002"  42.3517   10
  10  "classic"     "A32003"  42.3504   10
  11  "classic"     "A32004"  42.3386   10
  12  "classic"     "B32002"  42.3362   10
   ⋮             ⋱                     
 575  "classic"  …  "T32014"  42.5209  136
 576  "classic"     "F32001"  42.4117  139
 577  "classic"     "F32002"  42.4177  139
 578  "classic"     "F32003"  42.4017  139
 582  "classic"     "M32081"  42.3731    8
 583  "classic"  …  "M32082"  42.363     8
 584  "classic"     "G32001"  42.4265  140
 585  "classic"     "G32002"  42.4239  140
 586  "classic"     "V32015"  42.4082  104

In [46]:
n_stations=length(stations_info[:,1])

449

## Let's build our input data matrices

In [47]:
# Capacities C_i
C=stations_info[:,9][:,:];

In [48]:
# Distances D_ij
D=Array{Float64}(undef,n_stations,n_stations)
for i=1:n_stations
    for j=1:n_stations
        D[i,j]=sqrt((stations_info[i,6]-stations_info[j,6])^2+(stations_info[i,14]-stations_info[j,14])^2)
    end
end
#D

In [49]:
# Feasibility X_ij
threshold=0.02
X=Array{Float64}(undef,n_stations,n_stations)
for i=1:n_stations
    for j=1:n_stations
        if D[i,j]<threshold
            X[i,j]=1
        else
            X[i,j]=0
        end
    end
end
#X

In [50]:
# Let's count the non-zero values in X
sum(X[i,12] > 0 for i in 1:n_stations)

41

In [51]:
# Demand d_ijt
using PyCall

py"""
import pickle
 
def load_pickle(fpath):
    with open(fpath, "rb") as f:
        data = pickle.load(f)
    return data
"""

load_pickle = py"load_pickle"

d = load_pickle("../data/parameters/demand_matrix.pkl");

In [52]:
# Number of vans
K=5
# Capacity of vans
S=20

20

In [53]:
# Initial number of bikes available y_i1
y1=Array{Int64}(undef,n_stations)
for i in 1:n_stations
    y1[i]=floor(C[i]/2)
end

## Simplified formulation

In [57]:
function first_model(C,D,X,y1,K,S,d,n_stations)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    M=1000000
    lambda=0.1 # to be tuned
    
    # Decision variables
    @variable(model, x[1:n_stations, 1:n_stations, 1:K, 1:24], Bin)
    @variable(model, 0 <= z[1:n_stations, 1:n_stations, 1:K, 1:24], Int)
    @variable(model, 0 <= y[1:n_stations, 1:24], Int)
    @variable(model, 0 <= w[1:n_stations, 1:n_stations, 1:24], Int)
    @variable(model, 0 <= u[1:n_stations, 1:n_stations, 1:24], Int)

    # Add constraints
    @constraint(model, [i in 1:n_stations, t in 1:24], y[i,t] <= C[i])
    @constraint(model, [i in 1:n_stations, t in 2:24], y[i,t] - y[i,t-1] == sum(w[j,i,t] for j in 1:n_stations)-sum(w[i,j,t] for j in 1:n_stations)+sum(z[i,j,k,t] for j in 1:n_stations, k in 1:K))
    @constraint(model, [i in 1:n_stations], y[i,1] == y1[i])
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 2:24], z[i,j,k,t] <= S)
    @constraint(model, [i in 1:n_stations, t in 2:24], -y[i,t] <= sum(w[j,i,t] for j in 1:n_stations) - sum(w[j,i,t] for j in 1:n_stations))
    @constraint(model, [i in 1:n_stations, t in 2:24], sum(w[j,i,t] for j in 1:n_stations) - sum(w[j,i,t] for j in 1:n_stations)<= C[i] - y[i,t])
    @constraint(model, [k in 1:K, t in 2:24], sum(x[i,j,k,t] for i in 1:n_stations, j in 1:n_stations) <= 1)
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 2:24], z[i,j,k,t] <= M*x[i,j,k,t])
    @constraint(model, [i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 2:24], x[i,j,k,t] <= X[i,j])

    # Set objective
    @objective(model, Min, sum(u[i,j,t] for i in 1:n_stations, j in 1:n_stations, t in 1:24)+lambda*sum(D[i,j]*x[i,j,k,t] for i in 1:n_stations, j in 1:n_stations, k in 1:K, t in 1:24))
    
    # Solve the model
    optimize!(model)
    
    # Print the solution
    println("Objective value: ", objective_value(model))
    return value.(x), objective_value(model)
end

first_model (generic function with 1 method)

In [58]:
x, obj=first_model(C,D,X,y1,K,S,d,n_stations)